# Parametric Validation of AI-Generated 3D Wheel Meshes Using Engineering Constraints

## Preprocessing and Feature Engineering Notebook

**Author:** Mahil Kattilparambath Ramakrishnan  
**Course:** QM 640 Data Analytics Capstone, Walsh College  
**Date:** February 2026  

---

## Objective

This notebook builds the complete engineered feature dataset from DeepWheel simulation results and 3D mesh files (STL). The goal is to compute parametric features across 7 engineering validation layers (Layer 0-6) that serve as quality gates for AI-generated wheel designs.

---

## The 7 Engineering Validation Layers

| Layer | Name | Description |
|-------|------|-------------|
| 0 | Data Integrity & Scale | Bounding box checks, unit scale validation |
| 1 | Mesh Validity | Geometric soundness: watertight, manifold, triangle quality |
| 2 | Feature Extractability | Confidence indicators for CAD feature extraction |
| 3 | Physics Plausibility | Modal analysis outputs, frequency ratios, stiffness proxies |
| 4 | Engineering Constraints | Pass/fail evaluation against design thresholds |
| 5 | Manufacturability | Heuristic indicators for CAD/manufacturing readiness |
| 6 | Analytics & ML Readiness | Normalized features, risk scoring, classification labels |

Each layer produces a **gate output** (`layerX_pass` or `layerX_status`) and a **failure reason string**.

---

## Important Disclaimers

1. **Manufacturability metrics are heuristic indicators**, not certified manufacturing approval. They provide directional guidance but do not replace formal DFM (Design for Manufacturing) analysis.

2. **Layer 2 feature extractability scores are confidence measures**, not guarantees of successful CAD feature recognition. They indicate the likelihood that automated tools can extract features, not that they will.

3. **Physics plausibility uses DeepWheel simulation outputs**. These are computational approximations and should be validated with physical testing for production use.

4. All thresholds are **conservative and documented**. They are based on typical automotive wheel specifications and can be adjusted for specific applications.

---

## Outputs

- `data/processed/deepwheel_features_full.csv` - Complete feature dataset
- `data/processed/deepwheel_data_dictionary.csv` - Column definitions
- `data/processed/quality_report.json` - Summary statistics and failure analysis
- `docs/figures/Figure1_dataset_preview.png` - First 10 rows preview
- `docs/figures/Figure2_folder_tree_structure.png` - Repository structure
- `docs/figures/Figure3_3d_wheel_preview.png` - Sample wheel renderings

In [1]:
# =============================================================================
# Cell 2: Imports and Environment Checks
# =============================================================================

import sys
import subprocess

# Check and install trimesh if missing
try:
    import trimesh
    print(f"trimesh version: {trimesh.__version__}")
except ImportError:
    print("Installing trimesh...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "trimesh", "-q"])
    import trimesh
    print(f"trimesh installed: {trimesh.__version__}")

# Check and install tqdm if missing
try:
    from tqdm import tqdm
    print("tqdm available")
except ImportError:
    print("Installing tqdm...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "tqdm", "-q"])
    from tqdm import tqdm
    print("tqdm installed")

# Core imports
import pandas as pd
import numpy as np
from pathlib import Path
import json
import math
import warnings
from collections import Counter

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 200)

print("\n" + "="*60)
print("Environment Ready")
print(f"Python: {sys.version}")
print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")
print("="*60)

trimesh version: 4.11.1
tqdm available

Environment Ready
Python: 3.11.14 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 18:30:03) [MSC v.1929 64 bit (AMD64)]
Pandas: 2.3.3
NumPy: 1.26.4


In [2]:
# =============================================================================
# Cell 3: Define Paths and Create Output Folders
# =============================================================================

# Define base paths (notebook is in notebooks/ folder)
NOTEBOOK_DIR = Path.cwd()
if NOTEBOOK_DIR.name == 'notebooks':
    PROJECT_ROOT = NOTEBOOK_DIR.parent
else:
    PROJECT_ROOT = NOTEBOOK_DIR

# Input paths
DATA_DIR = PROJECT_ROOT / 'data'
SIM_RESULTS_CSV = DATA_DIR / 'deepwheel_sim_results.csv'
STL_DIR = DATA_DIR / 'stl'
STEP_DIR = DATA_DIR / 'step'

# Output paths
PROCESSED_DIR = DATA_DIR / 'processed'
FIGURES_DIR = PROJECT_ROOT / 'docs' / 'figures'

# Output files
OUTPUT_FEATURES_CSV = PROCESSED_DIR / 'deepwheel_features_full.csv'
OUTPUT_DATA_DICT_CSV = PROCESSED_DIR / 'deepwheel_data_dictionary.csv'
OUTPUT_QUALITY_JSON = PROCESSED_DIR / 'quality_report.json'

# Figure outputs
FIG1_PREVIEW = FIGURES_DIR / 'Figure1_dataset_preview.png'
FIG2_TREE = FIGURES_DIR / 'Figure2_folder_tree_structure.png'
FIG3_WHEEL = FIGURES_DIR / 'Figure3_3d_wheel_preview.png'

# Create output directories
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

print("Project Structure:")
print(f"  Project Root: {PROJECT_ROOT}")
print(f"  Data Directory: {DATA_DIR}")
print(f"  STL Directory: {STL_DIR} (exists: {STL_DIR.exists()})")
print(f"  STEP Directory: {STEP_DIR} (exists: {STEP_DIR.exists()})")
print(f"  Processed Output: {PROCESSED_DIR} (created: {PROCESSED_DIR.exists()})")
print(f"  Figures Output: {FIGURES_DIR} (created: {FIGURES_DIR.exists()})")
print(f"\nSimulation CSV: {SIM_RESULTS_CSV} (exists: {SIM_RESULTS_CSV.exists()})")

Project Structure:
  Project Root: C:\Users\mahil.kr\GL\data-analytics-capstone
  Data Directory: C:\Users\mahil.kr\GL\data-analytics-capstone\data
  STL Directory: C:\Users\mahil.kr\GL\data-analytics-capstone\data\stl (exists: True)
  STEP Directory: C:\Users\mahil.kr\GL\data-analytics-capstone\data\step (exists: True)
  Processed Output: C:\Users\mahil.kr\GL\data-analytics-capstone\data\processed (created: True)
  Figures Output: C:\Users\mahil.kr\GL\data-analytics-capstone\docs\figures (created: True)

Simulation CSV: C:\Users\mahil.kr\GL\data-analytics-capstone\data\deepwheel_sim_results.csv (exists: True)


In [3]:
# =============================================================================
# Cell 4: Load Base Simulation CSV
# =============================================================================

# Load the simulation results
print("Loading simulation results...")
df_raw = pd.read_csv(SIM_RESULTS_CSV)

# Store original column names
original_columns = df_raw.columns.tolist()
print(f"Original columns: {original_columns}")

# Create normalized column name mapping
def normalize_column_name(col):
    """Convert column name to lowercase with underscores."""
    return col.lower().replace(' ', '_').replace('-', '_')

# Create the main dataframe with normalized names
df = df_raw.copy()
column_mapping = {col: normalize_column_name(col) for col in df.columns}
df = df.rename(columns=column_mapping)

# Verify required columns exist
required_columns = ['file_name', 'mass', 'mode7_freq', 'mode11_freq']
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    raise ValueError(f"Missing required columns: {missing_columns}\n"
                     f"Available columns: {df.columns.tolist()}\n"
                     f"Please ensure the CSV contains: file_name, Mass, Mode7 Freq, Mode11 Freq")

# Keep original column names as separate columns for reference
df['mass_original'] = df['mass']
df['mode7_freq_original'] = df['mode7_freq']
df['mode11_freq_original'] = df['mode11_freq']

print(f"\nDataset loaded successfully:")
print(f"  Total designs: {len(df)}")
print(f"  Columns: {df.columns.tolist()}")
print(f"\nBasic statistics:")
print(df[['mass', 'mode7_freq', 'mode11_freq']].describe())

Loading simulation results...
Original columns: ['file_name', 'Mass', 'Mode7 Freq', 'Mode11 Freq']

Dataset loaded successfully:
  Total designs: 904
  Columns: ['file_name', 'mass', 'mode7_freq', 'mode11_freq', 'mass_original', 'mode7_freq_original', 'mode11_freq_original']

Basic statistics:
             mass  mode7_freq  mode11_freq
count  904.000000  904.000000   904.000000
mean    20.514222  417.427495  1134.729746
std      1.142835   21.299897   113.825731
min     17.615000  366.075400   910.437200
25%     19.701300  402.080750  1052.335250
50%     20.527300  417.026000  1142.719500
75%     21.290100  432.166000  1215.616000
max     24.146400  479.600300  1512.935000


In [4]:
# =============================================================================
# Cell 5: Map file_name to STL/STEP Paths
# =============================================================================

def find_stl_path(file_name, stl_dir):
    """Find STL file path, trying common naming patterns."""
    # Primary pattern: direct match
    primary = stl_dir / f"{file_name}.stl"
    if primary.exists():
        return str(primary)
    
    # Try lowercase
    lower = stl_dir / f"{file_name.lower()}.stl"
    if lower.exists():
        return str(lower)
    
    return None

def find_step_path(file_name, step_dir):
    """Find STEP file path, trying .stp and .step extensions."""
    # Try .stp first (more common in this dataset)
    stp_path = step_dir / f"{file_name}.stp"
    if stp_path.exists():
        return str(stp_path)
    
    # Try .step
    step_path = step_dir / f"{file_name}.step"
    if step_path.exists():
        return str(step_path)
    
    # Try lowercase
    stp_lower = step_dir / f"{file_name.lower()}.stp"
    if stp_lower.exists():
        return str(stp_lower)
    
    return None

print("Mapping file names to STL/STEP paths...")

# Map paths
df['stl_path'] = df['file_name'].apply(lambda x: find_stl_path(x, STL_DIR))
df['step_path'] = df['file_name'].apply(lambda x: find_step_path(x, STEP_DIR))

# Create boolean flags
df['has_stl'] = df['stl_path'].notna()
df['has_step'] = df['step_path'].notna()

# Summary
stl_count = df['has_stl'].sum()
step_count = df['has_step'].sum()
total = len(df)

print(f"\nFile availability summary:")
print(f"  Total designs in CSV: {total}")
print(f"  Designs with STL: {stl_count} ({100*stl_count/total:.1f}%)")
print(f"  Designs with STEP: {step_count} ({100*step_count/total:.1f}%)")
print(f"  Designs missing STL: {total - stl_count}")
print(f"  Designs missing STEP: {total - step_count}")

# Show sample of missing files
missing_stl = df[~df['has_stl']]['file_name'].head(5).tolist()
if missing_stl:
    print(f"\nSample designs missing STL: {missing_stl}")

Mapping file names to STL/STEP paths...

File availability summary:
  Total designs in CSV: 904
  Designs with STL: 904 (100.0%)
  Designs with STEP: 904 (100.0%)
  Designs missing STL: 0
  Designs missing STEP: 0


In [5]:
# =============================================================================
# Cell 6: Layer 0 - Data Integrity & Scale Consistency
# =============================================================================

print("Computing Layer 0: Data Integrity & Scale Consistency")
print("="*60)

# Define thresholds for typical automotive wheel dimensions (in mm)
# Typical wheel diameter: 14-22 inches (355-559mm)
# Typical wheel width: 5-12 inches (127-305mm)
# Adding margin for bounding box (which includes spokes extending to rim)
BBOX_MIN_MM = 100  # Minimum dimension in mm
BBOX_MAX_MM = 600  # Maximum dimension in mm

print(f"Scale thresholds: {BBOX_MIN_MM}mm - {BBOX_MAX_MM}mm per axis")

# Initialize Layer 0 columns
df['bbox_x'] = np.nan
df['bbox_y'] = np.nan
df['bbox_z'] = np.nan
df['bbox_volume_proxy'] = np.nan
df['unit_scale_flag'] = np.nan
df['axis_orientation_flag'] = np.nan  # Not computed - documented below
df['layer0_pass'] = False
df['layer0_fail_reason'] = ''

# Process each design with STL
stl_indices = df[df['has_stl']].index
print(f"\nProcessing {len(stl_indices)} STL files for bounding box...")

for idx in tqdm(stl_indices, desc="Layer 0 - Bounding Box"):
    stl_path = df.loc[idx, 'stl_path']
    fail_reasons = []
    
    try:
        # Load mesh
        mesh = trimesh.load(stl_path)
        
        # Get bounding box dimensions
        bounds = mesh.bounds  # [[min_x, min_y, min_z], [max_x, max_y, max_z]]
        bbox = bounds[1] - bounds[0]  # [size_x, size_y, size_z]
        
        df.loc[idx, 'bbox_x'] = bbox[0]
        df.loc[idx, 'bbox_y'] = bbox[1]
        df.loc[idx, 'bbox_z'] = bbox[2]
        df.loc[idx, 'bbox_volume_proxy'] = bbox[0] * bbox[1] * bbox[2]
        
        # Check scale consistency
        # Assume dimensions are in mm (typical for CAD exports)
        scale_ok = all(BBOX_MIN_MM <= dim <= BBOX_MAX_MM for dim in bbox)
        df.loc[idx, 'unit_scale_flag'] = not scale_ok
        
        if not scale_ok:
            too_small = [f"{['x','y','z'][i]}={bbox[i]:.1f}" for i, dim in enumerate(bbox) if dim < BBOX_MIN_MM]
            too_large = [f"{['x','y','z'][i]}={bbox[i]:.1f}" for i, dim in enumerate(bbox) if dim > BBOX_MAX_MM]
            if too_small:
                fail_reasons.append(f"bbox_too_small({','.join(too_small)})")
            if too_large:
                fail_reasons.append(f"bbox_too_large({','.join(too_large)})")
        
        # Axis orientation check: Z-axis should be the smallest dimension (wheel width)
        # For a properly oriented wheel: X ≈ Y (diameter) > Z (width)
        min_dim_axis = np.argmin(bbox)  # 0=X, 1=Y, 2=Z
        orientation_ok = (min_dim_axis == 2)  # Z should be smallest (axle direction)
        df.loc[idx, 'axis_orientation_flag'] = not orientation_ok  # True = problem
        
        if not orientation_ok:
            axis_names = ['X', 'Y', 'Z']
            fail_reasons.append(f"wrong_axis_orientation(smallest={axis_names[min_dim_axis]})")
        
        # Gate logic: pass if scale OK AND orientation OK
        layer0_pass = scale_ok and orientation_ok
        df.loc[idx, 'layer0_pass'] = layer0_pass
        df.loc[idx, 'layer0_fail_reason'] = '; '.join(fail_reasons) if fail_reasons else ''
        
    except Exception as e:
        df.loc[idx, 'layer0_pass'] = False
        df.loc[idx, 'layer0_fail_reason'] = f"mesh_load_error: {str(e)[:50]}"

# Handle designs without STL
no_stl_mask = ~df['has_stl']
df.loc[no_stl_mask, 'layer0_pass'] = False
df.loc[no_stl_mask, 'layer0_fail_reason'] = 'no_stl_file'

# Summary
layer0_pass_count = df['layer0_pass'].sum()
print(f"\nLayer 0 Summary:")
print(f"  Passed: {layer0_pass_count} ({100*layer0_pass_count/len(df):.1f}%)")
print(f"  Failed: {len(df) - layer0_pass_count}")
print(f"\nBounding box statistics (mm):")
print(df[['bbox_x', 'bbox_y', 'bbox_z']].describe())

# Summary of axis orientation
orientation_issues = df['axis_orientation_flag'].sum()
print(f"\nAxis orientation check:")
print(f"  Correctly oriented (Z is smallest): {len(df) - orientation_issues}")
print(f"  Incorrectly oriented: {int(orientation_issues)}")
print("  Heuristic: Z-axis should be wheel width (smallest bbox dimension)")
print("  X and Y should be wheel diameter (larger, roughly equal)")

Computing Layer 0: Data Integrity & Scale Consistency
Scale thresholds: 100mm - 600mm per axis

Processing 904 STL files for bounding box...


Layer 0 - Bounding Box: 100%|████████████████████████████████████████████████████████| 904/904 [01:27<00:00, 10.34it/s]


Layer 0 Summary:
  Passed: 904 (100.0%)
  Failed: 0

Bounding box statistics (mm):
           bbox_x      bbox_y      bbox_z
count  904.000000  904.000000  904.000000
mean   482.600000  482.600000  215.900000
std      0.000008    0.000008    0.000004
min    482.599991  482.599991  215.899994
25%    482.599991  482.599991  215.899994
50%    482.600006  482.600006  215.900002
75%    482.600006  482.600006  215.900002
max    482.600006  482.600006  215.900009

Axis orientation check:
  Correctly oriented (Z is smallest): 904
  Incorrectly oriented: 0
  Heuristic: Z-axis should be wheel width (smallest bbox dimension)
  X and Y should be wheel diameter (larger, roughly equal)


In [6]:
# =============================================================================
# Cell 7: Layer 1 - Mesh Validity (Geometric Soundness)
# =============================================================================

print("Computing Layer 1: Mesh Validity (Geometric Soundness)")
print("="*60)

# Layer 1 uses percentile-based thresholds (adaptive to dataset)
# Since AI-generated meshes often have quality issues, we use soft thresholds
FAIL_PERCENTILE_L1 = 25  # Bottom 25% on quality metrics will fail

print(f"Using percentile-based thresholds (adaptive to AI-generated mesh quality):")
print(f"  Aspect ratio: top {100-FAIL_PERCENTILE_L1}% (highest = worst) will be flagged")
print(f"  Watertight: informational only (many AI meshes are not watertight)")
print(f"  Non-manifold edges: informational only")

# Initialize Layer 1 columns
df['triangle_count'] = np.nan
df['is_watertight'] = np.nan
df['non_manifold_edge_count'] = np.nan
df['self_intersection_flag'] = np.nan  # Limited support in trimesh
df['normal_consistency_flag'] = np.nan
df['edge_length_mean'] = np.nan
df['edge_length_std'] = np.nan
df['triangle_aspect_ratio_max'] = np.nan
df['layer1_pass'] = False
df['layer1_fail_reason'] = ''

def compute_triangle_aspect_ratios(mesh):
    """Compute aspect ratio for each triangle (longest edge / shortest edge)."""
    vertices = mesh.vertices
    faces = mesh.faces
    
    # Get edge lengths for each triangle
    v0 = vertices[faces[:, 0]]
    v1 = vertices[faces[:, 1]]
    v2 = vertices[faces[:, 2]]
    
    edge1 = np.linalg.norm(v1 - v0, axis=1)
    edge2 = np.linalg.norm(v2 - v1, axis=1)
    edge3 = np.linalg.norm(v0 - v2, axis=1)
    
    edges = np.column_stack([edge1, edge2, edge3])
    min_edges = np.min(edges, axis=1)
    max_edges = np.max(edges, axis=1)
    
    # Avoid division by zero
    min_edges = np.maximum(min_edges, 1e-10)
    aspect_ratios = max_edges / min_edges
    
    return aspect_ratios

def count_non_manifold_edges(mesh):
    """Count edges that are not shared by exactly 2 faces."""
    try:
        # Get face adjacency - edges shared by != 2 faces are non-manifold
        edges = mesh.edges_unique
        edge_faces = mesh.edges_unique_inverse
        
        # Count faces per edge using edge_to_faces mapping
        edge_face_count = np.bincount(edge_faces, minlength=len(edges))
        
        # Non-manifold edges are those shared by != 2 faces
        non_manifold = np.sum((edge_face_count != 2) & (edge_face_count > 0))
        return int(non_manifold)
    except:
        return np.nan

# Process each design with STL
stl_indices = df[df['has_stl']].index
print(f"\nProcessing {len(stl_indices)} STL files for mesh validity...")

for idx in tqdm(stl_indices, desc="Layer 1 - Mesh Validity"):
    stl_path = df.loc[idx, 'stl_path']
    fail_reasons = []
    
    try:
        # Load mesh
        mesh = trimesh.load(stl_path)
        
        # Triangle count
        df.loc[idx, 'triangle_count'] = len(mesh.faces)
        
        # Watertight check
        is_watertight = mesh.is_watertight
        df.loc[idx, 'is_watertight'] = is_watertight
        if not is_watertight:
            fail_reasons.append('not_watertight')
        
        # Non-manifold edges
        non_manifold_count = count_non_manifold_edges(mesh)
        df.loc[idx, 'non_manifold_edge_count'] = non_manifold_count
        if non_manifold_count is not np.nan and non_manifold_count > 0:
            fail_reasons.append(f'non_manifold_edges({non_manifold_count})')
        
        # Self-intersection: Limited in trimesh, use broken_faces as proxy
        try:
            # Check for degenerate faces as a proxy
            degenerate = trimesh.repair.broken_faces(mesh)
            df.loc[idx, 'self_intersection_flag'] = len(degenerate) > 0
        except:
            df.loc[idx, 'self_intersection_flag'] = np.nan
        
        # Normal consistency (winding consistency)
        try:
            df.loc[idx, 'normal_consistency_flag'] = mesh.is_winding_consistent
        except:
            df.loc[idx, 'normal_consistency_flag'] = np.nan
        
        # Edge lengths
        edge_lengths = mesh.edges_unique_length
        df.loc[idx, 'edge_length_mean'] = np.mean(edge_lengths)
        df.loc[idx, 'edge_length_std'] = np.std(edge_lengths)
        
        # Triangle aspect ratio
        aspect_ratios = compute_triangle_aspect_ratios(mesh)
        max_aspect = np.max(aspect_ratios)
        df.loc[idx, 'triangle_aspect_ratio_max'] = max_aspect
        
    except Exception as e:
        # Mark as error - will be handled in gate logic below
        pass

# Handle designs without STL - set metrics to NaN
no_stl_mask = ~df['has_stl']

# Calculate percentile-based thresholds from computed metrics
print("\nCalculating percentile-based thresholds from data...")
ASPECT_RATIO_THRESHOLD = df['triangle_aspect_ratio_max'].quantile((100 - FAIL_PERCENTILE_L1) / 100)
print(f"  Aspect ratio threshold (P{100-FAIL_PERCENTILE_L1}): {ASPECT_RATIO_THRESHOLD:.1f}")
print(f"  Watertight rate: {df['is_watertight'].mean()*100:.1f}%")

# Apply gate logic based on percentile thresholds
print("\nApplying gate logic...")
for idx in df.index:
    fail_reasons = []
    
    if not df.loc[idx, 'has_stl']:
        df.loc[idx, 'layer1_pass'] = False
        df.loc[idx, 'layer1_fail_reason'] = 'no_stl_file'
        continue
    
    is_watertight = df.loc[idx, 'is_watertight']
    non_manifold = df.loc[idx, 'non_manifold_edge_count']
    max_aspect = df.loc[idx, 'triangle_aspect_ratio_max']
    
    # Check metrics against thresholds
    if pd.notna(is_watertight) and not is_watertight:
        fail_reasons.append('not_watertight')
    
    if pd.notna(non_manifold) and non_manifold > 0:
        fail_reasons.append(f'non_manifold_edges({int(non_manifold)})')
    
    if pd.notna(max_aspect) and max_aspect > ASPECT_RATIO_THRESHOLD:
        fail_reasons.append(f'aspect_ratio({max_aspect:.1f}>{ASPECT_RATIO_THRESHOLD:.1f})')
    
    # Gate logic: PASS if no more than 1 issue (lenient for AI-generated meshes)
    # This allows meshes that are not watertight but have good geometry to pass
    df.loc[idx, 'layer1_pass'] = len(fail_reasons) <= 1
    df.loc[idx, 'layer1_fail_reason'] = '; '.join(fail_reasons) if fail_reasons else ''

# Summary
layer1_pass_count = df['layer1_pass'].sum()
print(f"\nLayer 1 Summary:")
print(f"  Passed: {layer1_pass_count} ({100*layer1_pass_count/len(df):.1f}%)")
print(f"  Failed: {len(df) - layer1_pass_count}")
print(f"\nMesh statistics:")
print(f"  Watertight meshes: {df['is_watertight'].sum()}")
print(f"  Mean triangle count: {df['triangle_count'].mean():.0f}")
print(f"  Mean edge length: {df['edge_length_mean'].mean():.2f}")

# Note about self-intersection
print("\nNote: self_intersection_flag uses broken_faces() as a proxy.")
print("Full self-intersection detection requires more sophisticated algorithms.")

Computing Layer 1: Mesh Validity (Geometric Soundness)
Using percentile-based thresholds (adaptive to AI-generated mesh quality):
  Aspect ratio: top 75% (highest = worst) will be flagged
  Watertight: informational only (many AI meshes are not watertight)
  Non-manifold edges: informational only

Processing 904 STL files for mesh validity...


Layer 1 - Mesh Validity: 100%|███████████████████████████████████████████████████████| 904/904 [07:47<00:00,  1.93it/s]



Calculating percentile-based thresholds from data...
  Aspect ratio threshold (P75): 39.3
  Watertight rate: 5.5%

Applying gate logic...

Layer 1 Summary:
  Passed: 50 (5.5%)
  Failed: 854

Mesh statistics:
  Watertight meshes: 50
  Mean triangle count: 91681
  Mean edge length: 5.30

Note: self_intersection_flag uses broken_faces() as a proxy.
Full self-intersection detection requires more sophisticated algorithms.


In [7]:
# =============================================================================
# Cell 8: Layer 2 - Feature Extractability (Confidence Indicators)
# =============================================================================

print("Computing Layer 2: Feature Extractability (Confidence Indicators)")
print("="*60)

# Percentile-based thresholds (will be calculated from data)
# Designs below 25th percentile for symmetry OR above 75th percentile for RMSE/residual will fail
FAIL_PERCENTILE = 25  # Bottom 25% on each metric flagged as potential issues

print(f"Using percentile-based thresholds (adaptive to dataset):")
print(f"  Symmetry: bottom {FAIL_PERCENTILE}% will fail (lower = worse)")
print(f"  Flatness RMSE: top {100-FAIL_PERCENTILE}% will fail (higher = worse)")
print(f"  Roundness residual: top {100-FAIL_PERCENTILE}% will fail (higher = worse)")

# Initialize Layer 2 columns
df['symmetry_axis_confidence'] = np.nan
df['hub_plane_flatness_proxy'] = np.nan
df['center_bore_roundness_proxy'] = np.nan
df['bolt_hole_detectability_score'] = np.nan  # Not computed - too complex
df['layer2_status'] = 'UNKNOWN'
df['layer2_fail_reason'] = ''

def compute_symmetry_confidence(mesh):
    """Compute rotational symmetry confidence using PCA.
    Higher first eigenvalue ratio suggests strong alignment with one axis."""
    try:
        # Sample points from mesh surface
        points = mesh.vertices
        if len(points) > 5000:
            indices = np.random.choice(len(points), 5000, replace=False)
            points = points[indices]
        
        # Center points
        centered = points - np.mean(points, axis=0)
        
        # Compute covariance and eigenvalues
        cov = np.cov(centered.T)
        eigenvalues = np.linalg.eigvalsh(cov)
        eigenvalues = np.sort(eigenvalues)[::-1]  # Descending order
        
        # Confidence: ratio of largest eigenvalue to sum
        # For rotational symmetry (like a wheel), we expect one dominant axis
        confidence = eigenvalues[0] / np.sum(eigenvalues)
        return confidence
    except:
        return np.nan

def compute_hub_flatness(mesh):
    """Estimate hub plane flatness by fitting plane to points near min-Z region."""
    try:
        points = mesh.vertices
        z_coords = points[:, 2]
        
        # Find points in bottom 10% of Z range (hub region)
        z_range = z_coords.max() - z_coords.min()
        z_threshold = z_coords.min() + 0.1 * z_range
        hub_mask = z_coords <= z_threshold
        hub_points = points[hub_mask]
        
        if len(hub_points) < 10:
            return np.nan
        
        # Fit plane using least squares: z = ax + by + c
        A = np.column_stack([hub_points[:, 0], hub_points[:, 1], np.ones(len(hub_points))])
        b = hub_points[:, 2]
        
        # Solve least squares
        coeffs, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
        
        # Compute RMSE
        z_predicted = A @ coeffs
        rmse = np.sqrt(np.mean((b - z_predicted) ** 2))
        return rmse
    except:
        return np.nan

def compute_bore_roundness(mesh):
    """Estimate center bore roundness by fitting circle to hub slice."""
    try:
        points = mesh.vertices
        z_coords = points[:, 2]
        
        # Get points near hub (bottom 5% of Z)
        z_range = z_coords.max() - z_coords.min()
        z_threshold = z_coords.min() + 0.05 * z_range
        hub_mask = z_coords <= z_threshold
        hub_points = points[hub_mask]
        
        if len(hub_points) < 20:
            return np.nan
        
        # Find center bore: points near the center (inner 30% of radial distance)
        xy = hub_points[:, :2]
        center_approx = np.mean(xy, axis=0)
        radii = np.linalg.norm(xy - center_approx, axis=1)
        r_threshold = np.percentile(radii, 30)
        bore_mask = radii <= r_threshold
        bore_points = xy[bore_mask]
        
        if len(bore_points) < 10:
            return np.nan
        
        # Fit circle: minimize (r - R)^2 where r = sqrt((x-cx)^2 + (y-cy)^2)
        # Use mean as center, compute radius residuals
        center = np.mean(bore_points, axis=0)
        distances = np.linalg.norm(bore_points - center, axis=1)
        mean_radius = np.mean(distances)
        residual = np.sqrt(np.mean((distances - mean_radius) ** 2))
        return residual
    except:
        return np.nan

# Process each design with STL
stl_indices = df[df['has_stl']].index
print(f"\nProcessing {len(stl_indices)} STL files for feature extractability...")

for idx in tqdm(stl_indices, desc="Layer 2 - Feature Extractability"):
    stl_path = df.loc[idx, 'stl_path']
    fail_reasons = []
    
    try:
        mesh = trimesh.load(stl_path)
        
        # Symmetry confidence
        symmetry = compute_symmetry_confidence(mesh)
        df.loc[idx, 'symmetry_axis_confidence'] = symmetry
        
        # Hub flatness
        flatness = compute_hub_flatness(mesh)
        df.loc[idx, 'hub_plane_flatness_proxy'] = flatness
        
        # Bore roundness
        roundness = compute_bore_roundness(mesh)
        df.loc[idx, 'center_bore_roundness_proxy'] = roundness
        
        # Bolt hole detectability: Not computed (requires robust 2D projection analysis)
        df.loc[idx, 'bolt_hole_detectability_score'] = np.nan
        
    except Exception as e:
        # Mark as computation error - will be handled in gate logic below
        pass

# Calculate percentile-based thresholds from the computed metrics
print("\nCalculating percentile-based thresholds from data...")
SYMMETRY_THRESHOLD = df['symmetry_axis_confidence'].quantile(FAIL_PERCENTILE / 100)
FLATNESS_THRESHOLD = df['hub_plane_flatness_proxy'].quantile((100 - FAIL_PERCENTILE) / 100)
ROUNDNESS_THRESHOLD = df['center_bore_roundness_proxy'].quantile((100 - FAIL_PERCENTILE) / 100)

print(f"  Symmetry threshold (P{FAIL_PERCENTILE}): {SYMMETRY_THRESHOLD:.3f}")
print(f"  Flatness threshold (P{100-FAIL_PERCENTILE}): {FLATNESS_THRESHOLD:.2f} mm")
print(f"  Roundness threshold (P{100-FAIL_PERCENTILE}): {ROUNDNESS_THRESHOLD:.2f} mm")

# Apply gate logic based on percentile thresholds
print("\nApplying gate logic...")
for idx in df.index:
    symmetry = df.loc[idx, 'symmetry_axis_confidence']
    flatness = df.loc[idx, 'hub_plane_flatness_proxy']
    roundness = df.loc[idx, 'center_bore_roundness_proxy']
    
    fail_reasons = []
    
    # Check if all metrics are missing
    if pd.isna(symmetry) and pd.isna(flatness) and pd.isna(roundness):
        df.loc[idx, 'layer2_status'] = 'UNKNOWN'
        if not df.loc[idx, 'has_stl']:
            fail_reasons.append('no_stl_file')
        else:
            fail_reasons.append('all_metrics_unavailable')
    else:
        issues = []
        if not pd.isna(symmetry) and symmetry < SYMMETRY_THRESHOLD:
            issues.append(f'low_symmetry({symmetry:.2f}<{SYMMETRY_THRESHOLD:.2f})')
        if not pd.isna(flatness) and flatness > FLATNESS_THRESHOLD:
            issues.append(f'hub_not_flat({flatness:.1f}>{FLATNESS_THRESHOLD:.1f}mm)')
        if not pd.isna(roundness) and roundness > ROUNDNESS_THRESHOLD:
            issues.append(f'bore_not_round({roundness:.1f}>{ROUNDNESS_THRESHOLD:.1f}mm)')
        
        if issues:
            df.loc[idx, 'layer2_status'] = 'FAIL'
            fail_reasons.extend(issues)
        else:
            df.loc[idx, 'layer2_status'] = 'PASS'
    
    df.loc[idx, 'layer2_fail_reason'] = '; '.join(fail_reasons) if fail_reasons else ''

# Summary
status_counts = df['layer2_status'].value_counts()
print(f"\nLayer 2 Summary:")
for status, count in status_counts.items():
    print(f"  {status}: {count} ({100*count/len(df):.1f}%)")

print(f"\nFeature extractability statistics:")
print(f"  Mean symmetry confidence: {df['symmetry_axis_confidence'].mean():.3f}")
print(f"  Mean hub flatness RMSE: {df['hub_plane_flatness_proxy'].mean():.2f} mm")
print(f"  Mean bore roundness residual: {df['center_bore_roundness_proxy'].mean():.2f} mm")

print("\nNote: bolt_hole_detectability_score is set to NaN (not computed).")
print("Reason: Detecting bolt holes requires robust 2D projection and void detection.")
print("This is beyond the scope of simple heuristic computation.")

Computing Layer 2: Feature Extractability (Confidence Indicators)
Using percentile-based thresholds (adaptive to dataset):
  Symmetry: bottom 25% will fail (lower = worse)
  Flatness RMSE: top 75% will fail (higher = worse)
  Roundness residual: top 75% will fail (higher = worse)

Processing 904 STL files for feature extractability...


Layer 2 - Feature Extractability: 100%|██████████████████████████████████████████████| 904/904 [01:17<00:00, 11.64it/s]



Calculating percentile-based thresholds from data...
  Symmetry threshold (P25): 0.445
  Flatness threshold (P75): 6.44 mm
  Roundness threshold (P75): 8.77 mm

Applying gate logic...

Layer 2 Summary:
  FAIL: 550 (60.8%)
  PASS: 354 (39.2%)

Feature extractability statistics:
  Mean symmetry confidence: 0.449
  Mean hub flatness RMSE: 6.41 mm
  Mean bore roundness residual: 8.14 mm

Note: bolt_hole_detectability_score is set to NaN (not computed).
Reason: Detecting bolt holes requires robust 2D projection and void detection.
This is beyond the scope of simple heuristic computation.


In [8]:
# =============================================================================
# Cell 9: Layer 3 - Physics Plausibility (from DeepWheel outputs)
# =============================================================================

print("Computing Layer 3: Physics Plausibility")
print("="*60)

# Define physics thresholds
MODE7_THRESHOLD = 250  # Hz - NVH avoidance threshold
MASS_MAX = 25  # kg - typical passenger vehicle wheel limit
RATIO_MIN = 2.5  # minimum frequency ratio for modal separation

print(f"Thresholds:")
print(f"  Mode7 frequency minimum: {MODE7_THRESHOLD} Hz (NVH avoidance)")
print(f"  Maximum mass: {MASS_MAX} kg")
print(f"  Minimum frequency ratio (Mode11/Mode7): {RATIO_MIN}")

# Compute derived physics features
df['modal_separation'] = df['mode11_freq'] - df['mode7_freq']
df['frequency_ratio'] = df['mode11_freq'] / df['mode7_freq']

# Stiffness proxy: k = m * (2*pi*f)^2
df['stiffness_proxy_k7'] = df['mass'] * (2 * np.pi * df['mode7_freq']) ** 2
df['stiffness_proxy_k11'] = df['mass'] * (2 * np.pi * df['mode11_freq']) ** 2

# NVH margin
df['nvh_margin'] = df['mode7_freq'] - MODE7_THRESHOLD

# Layer 3 gate logic
df['layer3_mass_ok'] = df['mass'] <= MASS_MAX
df['layer3_mode7_ok'] = df['mode7_freq'] >= MODE7_THRESHOLD
df['layer3_ratio_ok'] = df['frequency_ratio'] >= RATIO_MIN

# Build failure reasons
def build_layer3_fail_reason(row):
    reasons = []
    if not row['layer3_mass_ok']:
        reasons.append(f"mass_exceeded({row['mass']:.2f}kg)")
    if not row['layer3_mode7_ok']:
        reasons.append(f"mode7_below_threshold({row['mode7_freq']:.1f}Hz)")
    if not row['layer3_ratio_ok']:
        reasons.append(f"ratio_below_min({row['frequency_ratio']:.2f})")
    return '; '.join(reasons)

df['layer3_pass'] = df['layer3_mass_ok'] & df['layer3_mode7_ok'] & df['layer3_ratio_ok']
df['layer3_fail_reason'] = df.apply(build_layer3_fail_reason, axis=1)

# Summary
layer3_pass_count = df['layer3_pass'].sum()
print(f"\nLayer 3 Summary:")
print(f"  Passed: {layer3_pass_count} ({100*layer3_pass_count/len(df):.1f}%)")
print(f"  Failed: {len(df) - layer3_pass_count}")

print(f"\nIndividual constraint pass rates:")
print(f"  Mass OK (≤{MASS_MAX}kg): {df['layer3_mass_ok'].sum()} ({100*df['layer3_mass_ok'].mean():.1f}%)")
print(f"  Mode7 OK (≥{MODE7_THRESHOLD}Hz): {df['layer3_mode7_ok'].sum()} ({100*df['layer3_mode7_ok'].mean():.1f}%)")
print(f"  Ratio OK (≥{RATIO_MIN}): {df['layer3_ratio_ok'].sum()} ({100*df['layer3_ratio_ok'].mean():.1f}%)")

print(f"\nPhysics statistics:")
print(df[['modal_separation', 'frequency_ratio', 'stiffness_proxy_k7', 'nvh_margin']].describe())

Computing Layer 3: Physics Plausibility
Thresholds:
  Mode7 frequency minimum: 250 Hz (NVH avoidance)
  Maximum mass: 25 kg
  Minimum frequency ratio (Mode11/Mode7): 2.5

Layer 3 Summary:
  Passed: 784 (86.7%)
  Failed: 120

Individual constraint pass rates:
  Mass OK (≤25kg): 904 (100.0%)
  Mode7 OK (≥250Hz): 904 (100.0%)
  Ratio OK (≥2.5): 784 (86.7%)

Physics statistics:
       modal_separation  frequency_ratio  stiffness_proxy_k7  nvh_margin
count        904.000000       904.000000        9.040000e+02  904.000000
mean         717.302251         2.712667        1.422377e+08  167.427495
std           94.669010         0.156769        2.223034e+07   21.299897
min          541.043200         2.383780        9.651935e+07  116.075400
25%          647.063450         2.595074        1.251201e+08  152.080750
50%          724.682800         2.731676        1.409280e+08  167.026000
75%          785.047875         2.826512        1.565976e+08  182.166000
max         1039.991300         3.20232

In [9]:
# =============================================================================
# Cell 10: Layer 4 - Engineering Constraint Evaluation
# =============================================================================

print("Computing Layer 4: Engineering Constraint Evaluation")
print("="*60)

# Define stiffness threshold based on percentile of high-quality designs
# Use 10th percentile of stiffness among designs that pass Layer 3
quality_subset = df[df['layer3_pass']]['stiffness_proxy_k7']
if len(quality_subset) > 0:
    STIFFNESS_THRESHOLD = quality_subset.quantile(0.10)
else:
    # Fallback: use 10th percentile of all designs
    STIFFNESS_THRESHOLD = df['stiffness_proxy_k7'].quantile(0.10)

print(f"Thresholds:")
print(f"  Mass: ≤ {MASS_MAX} kg")
print(f"  Mode7 frequency: ≥ {MODE7_THRESHOLD} Hz")
print(f"  Frequency ratio: ≥ {RATIO_MIN}")
print(f"  Stiffness (k7): ≥ {STIFFNESS_THRESHOLD:.0f} N/m (10th percentile of quality subset)")

# Compute constraint flags
df['mass_ok'] = df['mass'] <= MASS_MAX
df['mode7_ok'] = df['mode7_freq'] >= MODE7_THRESHOLD
df['ratio_ok'] = df['frequency_ratio'] >= RATIO_MIN
df['stiffness_ok'] = df['stiffness_proxy_k7'] >= STIFFNESS_THRESHOLD

# Aggregate metrics
constraint_cols = ['mass_ok', 'mode7_ok', 'ratio_ok', 'stiffness_ok']
df['constraint_violation_count'] = (~df[constraint_cols]).sum(axis=1)
df['passes_all_constraints'] = df[constraint_cols].all(axis=1)

# Build failure reasons
def build_layer4_fail_reason(row):
    reasons = []
    if not row['mass_ok']:
        reasons.append(f"mass({row['mass']:.2f}>{MASS_MAX})")
    if not row['mode7_ok']:
        reasons.append(f"mode7({row['mode7_freq']:.1f}<{MODE7_THRESHOLD})")
    if not row['ratio_ok']:
        reasons.append(f"ratio({row['frequency_ratio']:.2f}<{RATIO_MIN})")
    if not row['stiffness_ok']:
        reasons.append(f"stiffness({row['stiffness_proxy_k7']:.0f}<{STIFFNESS_THRESHOLD:.0f})")
    return '; '.join(reasons)

df['layer4_pass'] = df['passes_all_constraints']
df['layer4_fail_reason'] = df.apply(build_layer4_fail_reason, axis=1)

# Summary
layer4_pass_count = df['layer4_pass'].sum()
print(f"\nLayer 4 Summary:")
print(f"  Passed all constraints: {layer4_pass_count} ({100*layer4_pass_count/len(df):.1f}%)")
print(f"  Failed at least one: {len(df) - layer4_pass_count}")

print(f"\nViolation count distribution:")
violation_dist = df['constraint_violation_count'].value_counts().sort_index()
for count, n in violation_dist.items():
    print(f"  {count} violations: {n} designs ({100*n/len(df):.1f}%)")

print(f"\nIndividual constraint pass rates:")
for col in constraint_cols:
    print(f"  {col}: {df[col].sum()} ({100*df[col].mean():.1f}%)")

Computing Layer 4: Engineering Constraint Evaluation
Thresholds:
  Mass: ≤ 25 kg
  Mode7 frequency: ≥ 250 Hz
  Frequency ratio: ≥ 2.5
  Stiffness (k7): ≥ 121697716 N/m (10th percentile of quality subset)

Layer 4 Summary:
  Passed all constraints: 705 (78.0%)
  Failed at least one: 199

Violation count distribution:
  0 violations: 705 designs (78.0%)
  1 violations: 108 designs (11.9%)
  2 violations: 91 designs (10.1%)

Individual constraint pass rates:
  mass_ok: 904 (100.0%)
  mode7_ok: 904 (100.0%)
  ratio_ok: 784 (86.7%)
  stiffness_ok: 734 (81.2%)


In [10]:
# =============================================================================
# Cell 11: Layer 5 - Manufacturability & CAD Readiness (Heuristics)
# =============================================================================

print("Computing Layer 5: Manufacturability & CAD Readiness")
print("="*60)

# Thresholds
DIHEDRAL_ANGLE_THRESHOLD = 150  # degrees, edges above this are "sharp"
SHARP_EDGE_DENSITY_MAX = 0.1  # max acceptable sharp edges per mm^2

print(f"Thresholds:")
print(f"  Dihedral angle for sharp edge: > {DIHEDRAL_ANGLE_THRESHOLD} degrees")
print(f"  Max sharp edge density: {SHARP_EDGE_DENSITY_MAX} edges/mm^2")

# Initialize Layer 5 columns
df['min_wall_thickness_est'] = np.nan  # Complex computation
df['sharp_edge_density'] = np.nan
df['fillet_feasibility_score'] = np.nan
df['undercut_risk_score'] = np.nan  # Not computed
df['balance_uniformity'] = np.nan
df['cad_readiness_score'] = np.nan
df['layer5_status'] = 'UNKNOWN'
df['layer5_reason'] = ''

def compute_sharp_edge_density(mesh):
    """Compute sharp edge density: count edges with dihedral angle > threshold."""
    try:
        # Get face adjacency for dihedral angles
        face_adjacency = mesh.face_adjacency
        face_adjacency_angles = mesh.face_adjacency_angles
        
        # Convert to degrees
        angles_deg = np.degrees(face_adjacency_angles)
        
        # Count sharp edges (dihedral angle > threshold)
        sharp_edges = np.sum(angles_deg > DIHEDRAL_ANGLE_THRESHOLD)
        
        # Normalize by surface area
        surface_area = mesh.area
        if surface_area > 0:
            density = sharp_edges / surface_area
        else:
            density = np.nan
        
        return density, sharp_edges
    except:
        return np.nan, np.nan

def compute_balance_uniformity(mesh):
    """Compute balance uniformity using radial mass distribution proxy."""
    try:
        points = mesh.vertices
        
        # Find approximate axis (assume Z is rotation axis)
        center = np.mean(points, axis=0)
        
        # Compute radial distances from center (in XY plane)
        xy = points[:, :2] - center[:2]
        radii = np.linalg.norm(xy, axis=1)
        
        # Coefficient of variation of radial distances
        if np.mean(radii) > 0:
            cv = np.std(radii) / np.mean(radii)
        else:
            cv = np.nan
        
        return cv
    except:
        return np.nan

def compute_wall_thickness_proxy(mesh):
    """Estimate minimum wall thickness using KDTree nearest neighbor."""
    try:
        from scipy.spatial import KDTree
        
        # Sample surface points
        points = mesh.vertices
        normals = mesh.vertex_normals
        
        if len(points) > 2000:
            indices = np.random.choice(len(points), 2000, replace=False)
            points = points[indices]
            normals = normals[indices]
        
        # Build KDTree
        tree = KDTree(points)
        
        # For each point, find nearest point in opposite direction
        # (approximate: find nearest point not too close in normal direction)
        thicknesses = []
        for i, (p, n) in enumerate(zip(points, normals)):
            # Query multiple nearest neighbors
            distances, indices = tree.query(p, k=min(20, len(points)))
            
            # Find nearest point that's in roughly opposite normal direction
            for d, idx in zip(distances[1:], indices[1:]):  # Skip self
                if d > 0:
                    direction = points[idx] - p
                    direction = direction / np.linalg.norm(direction)
                    # Check if opposite direction (dot product with normal < 0)
                    if np.dot(direction, n) < -0.5:  # Roughly opposite
                        thicknesses.append(d)
                        break
        
        if thicknesses:
            return np.min(thicknesses)
        else:
            return np.nan
    except:
        return np.nan

# Process each design with STL
stl_indices = df[df['has_stl']].index
print(f"\nProcessing {len(stl_indices)} STL files for manufacturability...")

for idx in tqdm(stl_indices, desc="Layer 5 - Manufacturability"):
    stl_path = df.loc[idx, 'stl_path']
    fail_reasons = []
    
    try:
        mesh = trimesh.load(stl_path)
        
        # Wall thickness (simplified proxy)
        thickness = compute_wall_thickness_proxy(mesh)
        df.loc[idx, 'min_wall_thickness_est'] = thickness
        
        # Sharp edge density
        density, _ = compute_sharp_edge_density(mesh)
        df.loc[idx, 'sharp_edge_density'] = density
        
        # Fillet feasibility (inverse of sharp edge density)
        if not pd.isna(density) and density > 0:
            # Normalize: 1 means no sharp edges, 0 means max sharp edges
            fillet_score = max(0, 1 - density / SHARP_EDGE_DENSITY_MAX)
            df.loc[idx, 'fillet_feasibility_score'] = fillet_score
        
        # Undercut risk: Not computed (requires visibility analysis)
        df.loc[idx, 'undercut_risk_score'] = np.nan
        
        # Balance uniformity
        uniformity = compute_balance_uniformity(mesh)
        df.loc[idx, 'balance_uniformity'] = uniformity
        
        # CAD readiness score: weighted average with nuanced mesh validity
        # Weights: mesh validity (0.25), feature extractability (0.35), manufacturability (0.40)
        # Use nuanced mesh score (accounts for AI mesh quality issues)
        mesh_issues = 0
        if not df.loc[idx, 'is_watertight']:
            mesh_issues += 1
        if df.loc[idx, 'non_manifold_edge_count'] > 0:
            mesh_issues += 1
        mesh_valid = max(0.3, 1.0 - (mesh_issues * 0.25))  # Min 0.3 for AI meshes
        
        feat_extract = 0.5  # Default neutral
        if df.loc[idx, 'layer2_status'] == 'PASS':
            feat_extract = 1.0
        elif df.loc[idx, 'layer2_status'] == 'FAIL':
            feat_extract = 0.35  # Not 0, since these are soft indicators
        
        manuf_score = df.loc[idx, 'fillet_feasibility_score']
        if pd.isna(manuf_score):
            manuf_score = 0.5  # Neutral if unknown
        
        cad_score = 0.25 * mesh_valid + 0.35 * feat_extract + 0.40 * manuf_score
        df.loc[idx, 'cad_readiness_score'] = cad_score
        
        # Gate logic - adjusted thresholds for AI-generated meshes
        if cad_score >= 0.50:
            df.loc[idx, 'layer5_status'] = 'PASS'
        elif cad_score >= 0.35:
            df.loc[idx, 'layer5_status'] = 'UNKNOWN'
            fail_reasons.append(f'marginal_cad_score({cad_score:.2f})')
        else:
            df.loc[idx, 'layer5_status'] = 'FAIL'
            fail_reasons.append(f'low_cad_score({cad_score:.2f})')
        
        df.loc[idx, 'layer5_reason'] = '; '.join(fail_reasons) if fail_reasons else ''
        
    except Exception as e:
        df.loc[idx, 'layer5_status'] = 'UNKNOWN'
        df.loc[idx, 'layer5_reason'] = f"computation_error: {str(e)[:50]}"

# Handle designs without STL
no_stl_mask = ~df['has_stl']
df.loc[no_stl_mask, 'layer5_status'] = 'UNKNOWN'
df.loc[no_stl_mask, 'layer5_reason'] = 'no_stl_file'

# Summary
status_counts = df['layer5_status'].value_counts()
print(f"\nLayer 5 Summary:")
for status, count in status_counts.items():
    print(f"  {status}: {count} ({100*count/len(df):.1f}%)")

print(f"\nManufacturability statistics:")
print(f"  Mean wall thickness est: {df['min_wall_thickness_est'].mean():.2f} mm")
print(f"  Mean sharp edge density: {df['sharp_edge_density'].mean():.6f}")
print(f"  Mean CAD readiness score: {df['cad_readiness_score'].mean():.2f}")

print("\nNote: undercut_risk_score is set to NaN (not computed).")
print("Reason: Undercut detection requires visibility analysis from mold direction.")
print("This is beyond the scope of simple mesh-based heuristics.")

Computing Layer 5: Manufacturability & CAD Readiness
Thresholds:
  Dihedral angle for sharp edge: > 150 degrees
  Max sharp edge density: 0.1 edges/mm^2

Processing 904 STL files for manufacturability...


Layer 5 - Manufacturability: 100%|███████████████████████████████████████████████████| 904/904 [04:10<00:00,  3.60it/s]


Layer 5 Summary:
  PASS: 884 (97.8%)
  UNKNOWN: 20 (2.2%)

Manufacturability statistics:
  Mean wall thickness est: 4.18 mm
  Mean sharp edge density: 0.000029
  Mean CAD readiness score: 0.73

Note: undercut_risk_score is set to NaN (not computed).
Reason: Undercut detection requires visibility analysis from mold direction.
This is beyond the scope of simple mesh-based heuristics.


In [11]:
# =============================================================================
# Cell 12: Layer 6 - Analytics & ML Readiness
# =============================================================================

print("Computing Layer 6: Analytics & ML Readiness")
print("="*60)

# Risk level thresholds - using percentile-based approach for balanced distribution
# Will be calculated after design_margin_score is computed
print("Risk levels will be assigned using percentile-based thresholds")

# Normalized features (min-max normalization)
df['normalized_mass'] = (df['mass'] - df['mass'].min()) / (df['mass'].max() - df['mass'].min())
df['normalized_mode7'] = (df['mode7_freq'] - df['mode7_freq'].min()) / (df['mode7_freq'].max() - df['mode7_freq'].min())

# Z-score normalization
df['z_mass'] = (df['mass'] - df['mass'].mean()) / df['mass'].std()
df['z_mode7'] = (df['mode7_freq'] - df['mode7_freq'].mean()) / df['mode7_freq'].std()
df['z_mode11'] = (df['mode11_freq'] - df['mode11_freq'].mean()) / df['mode11_freq'].std()
df['z_frequency_ratio'] = (df['frequency_ratio'] - df['frequency_ratio'].mean()) / df['frequency_ratio'].std()

# Margin calculations (normalized distance to constraint boundary)
# Higher margin = better (further from failure)
df['mass_margin_norm'] = (MASS_MAX - df['mass']) / MASS_MAX
df['mode7_margin_norm'] = (df['mode7_freq'] - MODE7_THRESHOLD) / MODE7_THRESHOLD
df['ratio_margin_norm'] = (df['frequency_ratio'] - RATIO_MIN) / RATIO_MIN

# Stiffness margin (normalized to threshold)
df['stiffness_margin_norm'] = (df['stiffness_proxy_k7'] - STIFFNESS_THRESHOLD) / STIFFNESS_THRESHOLD

# Design margin score: minimum of all normalized margins
margin_cols = ['mass_margin_norm', 'mode7_margin_norm', 'ratio_margin_norm', 'stiffness_margin_norm']
df['design_margin_score'] = df[margin_cols].min(axis=1)

# Risk level classification using percentile-based thresholds
# This ensures a balanced distribution across risk categories
RISK_HIGH_THRESHOLD = df['design_margin_score'].quantile(0.33)  # Bottom 33%
RISK_MEDIUM_THRESHOLD = df['design_margin_score'].quantile(0.67)  # Middle 33%

print(f"\nPercentile-based risk thresholds:")
print(f"  High risk: margin < {RISK_HIGH_THRESHOLD:.3f} (P33)")
print(f"  Medium risk: {RISK_HIGH_THRESHOLD:.3f} ≤ margin < {RISK_MEDIUM_THRESHOLD:.3f} (P33-P67)")
print(f"  Low risk: margin ≥ {RISK_MEDIUM_THRESHOLD:.3f} (P67+)")

def classify_risk(margin):
    if pd.isna(margin):
        return 'UNKNOWN'
    elif margin < RISK_HIGH_THRESHOLD:
        return 'High'
    elif margin < RISK_MEDIUM_THRESHOLD:
        return 'Medium'
    else:
        return 'Low'

df['risk_level'] = df['design_margin_score'].apply(classify_risk)

# Vehicle class label: Cannot determine without diameter
# Diameter is not directly available from mesh (would need consistent orientation)
# Set as UNKNOWN with documentation
df['vehicle_class_label'] = 'UNKNOWN'

# Summary
print(f"\nLayer 6 Summary:")
risk_dist = df['risk_level'].value_counts()
for level, count in risk_dist.items():
    print(f"  {level} risk: {count} ({100*count/len(df):.1f}%)")

print(f"\nDesign margin statistics:")
print(df['design_margin_score'].describe())

print(f"\nZ-score ranges:")
for col in ['z_mass', 'z_mode7', 'z_frequency_ratio']:
    print(f"  {col}: [{df[col].min():.2f}, {df[col].max():.2f}]")

print("\nNote: vehicle_class_label is set to UNKNOWN for all designs.")
print("Reason: Wheel diameter cannot be reliably extracted from mesh geometry alone.")
print("Proper vehicle class assignment requires additional metadata or consistent mesh orientation.")

Computing Layer 6: Analytics & ML Readiness
Risk levels will be assigned using percentile-based thresholds

Percentile-based risk thresholds:
  High risk: margin < 0.036 (P33)
  Medium risk: 0.036 ≤ margin < 0.105 (P33-P67)
  Low risk: margin ≥ 0.105 (P67+)

Layer 6 Summary:
  Medium risk: 308 (34.1%)
  Low risk: 298 (33.0%)
  High risk: 298 (33.0%)

Design margin statistics:
count    904.000000
mean       0.055062
std        0.077918
min       -0.206893
25%        0.010692
50%        0.078798
75%        0.115453
max        0.167055
Name: design_margin_score, dtype: float64

Z-score ranges:
  z_mass: [-2.54, 3.18]
  z_mode7: [-2.41, 2.92]
  z_frequency_ratio: [-2.10, 3.12]

Note: vehicle_class_label is set to UNKNOWN for all designs.
Reason: Wheel diameter cannot be reliably extracted from mesh geometry alone.
Proper vehicle class assignment requires additional metadata or consistent mesh orientation.


In [12]:
# =============================================================================
# Cell 13: Assemble Final Dataset + Ordering
# =============================================================================

print("Assembling final dataset...")
print("="*60)

# Define column ordering by layer
column_order = [
    # Original/Raw columns
    'file_name',
    'mass', 'mass_original',
    'mode7_freq', 'mode7_freq_original',
    'mode11_freq', 'mode11_freq_original',
    
    # File paths
    'stl_path', 'step_path', 'has_stl', 'has_step',
    
    # Layer 0: Data Integrity & Scale
    'bbox_x', 'bbox_y', 'bbox_z', 'bbox_volume_proxy',
    'unit_scale_flag', 'axis_orientation_flag',
    'layer0_pass', 'layer0_fail_reason',
    
    # Layer 1: Mesh Validity
    'triangle_count', 'is_watertight', 'non_manifold_edge_count',
    'self_intersection_flag', 'normal_consistency_flag',
    'edge_length_mean', 'edge_length_std', 'triangle_aspect_ratio_max',
    'layer1_pass', 'layer1_fail_reason',
    
    # Layer 2: Feature Extractability
    'symmetry_axis_confidence', 'hub_plane_flatness_proxy',
    'center_bore_roundness_proxy', 'bolt_hole_detectability_score',
    'layer2_status', 'layer2_fail_reason',
    
    # Layer 3: Physics Plausibility
    'modal_separation', 'frequency_ratio',
    'stiffness_proxy_k7', 'stiffness_proxy_k11', 'nvh_margin',
    'layer3_mass_ok', 'layer3_mode7_ok', 'layer3_ratio_ok',
    'layer3_pass', 'layer3_fail_reason',
    
    # Layer 4: Engineering Constraints
    'mass_ok', 'mode7_ok', 'ratio_ok', 'stiffness_ok',
    'constraint_violation_count', 'passes_all_constraints',
    'layer4_pass', 'layer4_fail_reason',
    
    # Layer 5: Manufacturability
    'min_wall_thickness_est', 'sharp_edge_density',
    'fillet_feasibility_score', 'undercut_risk_score', 'balance_uniformity',
    'cad_readiness_score', 'layer5_status', 'layer5_reason',
    
    # Layer 6: Analytics & ML Readiness
    'normalized_mass', 'normalized_mode7',
    'z_mass', 'z_mode7', 'z_mode11', 'z_frequency_ratio',
    'mass_margin_norm', 'mode7_margin_norm', 'ratio_margin_norm', 'stiffness_margin_norm',
    'design_margin_score', 'risk_level', 'vehicle_class_label'
]

# Verify all columns exist and add any missing ones
existing_cols = set(df.columns)
ordered_cols = [col for col in column_order if col in existing_cols]
missing_from_order = existing_cols - set(column_order)
missing_from_df = set(column_order) - existing_cols

if missing_from_order:
    print(f"Note: Columns not in ordering (will be appended): {missing_from_order}")
    ordered_cols.extend(sorted(missing_from_order))

if missing_from_df:
    print(f"Warning: Columns in ordering but not in dataframe: {missing_from_df}")

# Reorder columns
df_final = df[ordered_cols].copy()

# Save to CSV
df_final.to_csv(OUTPUT_FEATURES_CSV, index=False)
print(f"\nSaved feature dataset to: {OUTPUT_FEATURES_CSV}")
print(f"  Total rows: {len(df_final)}")
print(f"  Total columns: {len(df_final.columns)}")

# Show column groups
print(f"\nColumn groups:")
print(f"  Raw/Original: {len([c for c in ordered_cols if 'original' in c or c in ['file_name', 'mass', 'mode7_freq', 'mode11_freq']])}")
print(f"  File paths: {len([c for c in ordered_cols if 'path' in c or 'has_' in c])}")
print(f"  Layer 0: {len([c for c in ordered_cols if 'layer0' in c or 'bbox' in c or 'scale' in c or 'orientation' in c])}")
print(f"  Layer 1: {len([c for c in ordered_cols if 'layer1' in c or 'triangle' in c or 'watertight' in c or 'manifold' in c or 'edge_length' in c])}")
print(f"  Layer 2: {len([c for c in ordered_cols if 'layer2' in c or 'symmetry' in c or 'flatness' in c or 'roundness' in c or 'bolt_hole' in c])}")
print(f"  Layer 3: {len([c for c in ordered_cols if 'layer3' in c or 'modal' in c or 'frequency_ratio' in c or 'stiffness_proxy' in c or 'nvh' in c])}")
print(f"  Layer 4: {len([c for c in ordered_cols if 'layer4' in c or '_ok' in c or 'constraint' in c or 'passes_all' in c])}")
print(f"  Layer 5: {len([c for c in ordered_cols if 'layer5' in c or 'thickness' in c or 'sharp' in c or 'fillet' in c or 'undercut' in c or 'balance' in c or 'cad_' in c])}")
print(f"  Layer 6: {len([c for c in ordered_cols if 'normalized' in c or 'z_' in c or 'margin' in c or 'risk' in c or 'vehicle' in c])}")

Assembling final dataset...

Saved feature dataset to: C:\Users\mahil.kr\GL\data-analytics-capstone\data\processed\deepwheel_features_full.csv
  Total rows: 904
  Total columns: 74

Column groups:
  Raw/Original: 7
  File paths: 4
  Layer 0: 8
  Layer 1: 8
  Layer 2: 6
  Layer 3: 11
  Layer 4: 11
  Layer 5: 8
  Layer 6: 15


In [13]:
# =============================================================================
# Cell 14: Create Data Dictionary CSV
# =============================================================================

print("Creating data dictionary...")
print("="*60)

# Define data dictionary entries
data_dict_entries = [
    # Raw columns
    ('file_name', 'raw', 'Unique identifier for each wheel design', 'string', '-', 'raw', 'From DeepWheel simulation', 'alphanumeric string'),
    ('mass', 'raw', 'Wheel mass from simulation', 'float', 'kg', 'raw', 'Direct from DeepWheel', '15-30 kg typical'),
    ('mass_original', 'raw', 'Original mass value (backup)', 'float', 'kg', 'raw', 'Copy of mass column', '15-30 kg typical'),
    ('mode7_freq', 'raw', '7th mode natural frequency', 'float', 'Hz', 'raw', 'DeepWheel modal analysis', '300-500 Hz typical'),
    ('mode7_freq_original', 'raw', 'Original Mode7 frequency (backup)', 'float', 'Hz', 'raw', 'Copy of mode7_freq', '300-500 Hz typical'),
    ('mode11_freq', 'raw', '11th mode natural frequency', 'float', 'Hz', 'raw', 'DeepWheel modal analysis', '900-1400 Hz typical'),
    ('mode11_freq_original', 'raw', 'Original Mode11 frequency (backup)', 'float', 'Hz', 'raw', 'Copy of mode11_freq', '900-1400 Hz typical'),
    
    # File paths
    ('stl_path', 'raw', 'Path to STL mesh file', 'string', '-', 'derived', 'data/stl/{file_name}.stl', 'file path or null'),
    ('step_path', 'raw', 'Path to STEP CAD file', 'string', '-', 'derived', 'data/step/{file_name}.stp', 'file path or null'),
    ('has_stl', 'raw', 'Whether STL file exists', 'boolean', '-', 'derived', 'stl_path is not null', 'True/False'),
    ('has_step', 'raw', 'Whether STEP file exists', 'boolean', '-', 'derived', 'step_path is not null', 'True/False'),
    
    # Layer 0
    ('bbox_x', 0, 'Bounding box X dimension', 'float', 'mm', 'mesh', 'max(x) - min(x) from STL vertices', '100-600 mm expected'),
    ('bbox_y', 0, 'Bounding box Y dimension', 'float', 'mm', 'mesh', 'max(y) - min(y) from STL vertices', '100-600 mm expected'),
    ('bbox_z', 0, 'Bounding box Z dimension', 'float', 'mm', 'mesh', 'max(z) - min(z) from STL vertices', '100-600 mm expected'),
    ('bbox_volume_proxy', 0, 'Bounding box volume proxy', 'float', 'mm^3', 'mesh', 'bbox_x * bbox_y * bbox_z', 'varies'),
    ('unit_scale_flag', 0, 'Flag for out-of-scale dimensions', 'boolean', '-', 'derived', 'any bbox dim < 100 or > 600 mm', 'True=problem'),
    ('axis_orientation_flag', 0, 'Flag for incorrect axis orientation', 'boolean', '-', 'derived', 'True if Z is not the smallest bbox dimension', 'True=problem, False=OK'),
    ('layer0_pass', 0, 'Layer 0 gate pass status', 'boolean', '-', 'derived', 'unit_scale_flag == False AND axis_orientation_flag == False', 'True/False'),
    ('layer0_fail_reason', 0, 'Reason for Layer 0 failure', 'string', '-', 'derived', 'Concatenated failure reasons', 'text or empty'),
    
    # Layer 1
    ('triangle_count', 1, 'Number of triangles in mesh', 'int', '-', 'mesh', 'len(mesh.faces)', 'varies'),
    ('is_watertight', 1, 'Whether mesh is watertight', 'boolean', '-', 'mesh', 'mesh.is_watertight', 'True/False'),
    ('non_manifold_edge_count', 1, 'Count of non-manifold edges', 'int', '-', 'mesh', 'edges shared by != 2 faces', '0 is ideal'),
    ('self_intersection_flag', 1, 'Self-intersection proxy flag', 'boolean', '-', 'mesh', 'len(broken_faces) > 0', 'True/False or NaN'),
    ('normal_consistency_flag', 1, 'Normal winding consistency', 'boolean', '-', 'mesh', 'mesh.is_winding_consistent', 'True/False'),
    ('edge_length_mean', 1, 'Mean edge length', 'float', 'mm', 'mesh', 'mean(edges_unique_length)', 'varies'),
    ('edge_length_std', 1, 'Edge length standard deviation', 'float', 'mm', 'mesh', 'std(edges_unique_length)', 'varies'),
    ('triangle_aspect_ratio_max', 1, 'Maximum triangle aspect ratio', 'float', '-', 'mesh', 'max(longest_edge/shortest_edge)', '< 20 is good'),
    ('layer1_pass', 1, 'Layer 1 gate pass status', 'boolean', '-', 'derived', 'PASS if at most 1 issue (lenient for AI meshes)', 'True/False'),
    ('layer1_fail_reason', 1, 'Reason for Layer 1 failure', 'string', '-', 'derived', 'Concatenated failure reasons', 'text or empty'),
    
    # Layer 2
    ('symmetry_axis_confidence', 2, 'Rotational symmetry confidence', 'float', '-', 'mesh', 'PCA max eigenvalue / sum', '0-1, higher=better'),
    ('hub_plane_flatness_proxy', 2, 'Hub plane flatness RMSE', 'float', 'mm', 'mesh', 'RMSE of plane fit to hub region', '< 5mm is good'),
    ('center_bore_roundness_proxy', 2, 'Center bore roundness residual', 'float', 'mm', 'mesh', 'Circle fit residual at hub', '< 3mm is good'),
    ('bolt_hole_detectability_score', 2, 'Bolt hole detection confidence', 'float', '-', 'mesh', 'Not computed (NaN)', 'NaN'),
    ('layer2_status', 2, 'Layer 2 gate status', 'string', '-', 'derived', 'PASS/FAIL/UNKNOWN based on thresholds', 'PASS/FAIL/UNKNOWN'),
    ('layer2_fail_reason', 2, 'Reason for Layer 2 status', 'string', '-', 'derived', 'Concatenated issues', 'text or empty'),
    
    # Layer 3
    ('modal_separation', 3, 'Mode 11 - Mode 7 frequency gap', 'float', 'Hz', 'derived', 'mode11_freq - mode7_freq', '600-1000 Hz typical'),
    ('frequency_ratio', 3, 'Mode 11 / Mode 7 frequency ratio', 'float', '-', 'derived', 'mode11_freq / mode7_freq', '> 2.5 is good'),
    ('stiffness_proxy_k7', 3, 'Stiffness proxy using Mode 7', 'float', 'N/m (proxy)', 'derived', 'mass * (2*pi*mode7_freq)^2', 'varies'),
    ('stiffness_proxy_k11', 3, 'Stiffness proxy using Mode 11', 'float', 'N/m (proxy)', 'derived', 'mass * (2*pi*mode11_freq)^2', 'varies'),
    ('nvh_margin', 3, 'NVH margin above 250 Hz', 'float', 'Hz', 'derived', 'mode7_freq - 250', '> 0 is good'),
    ('layer3_mass_ok', 3, 'Mass within limit', 'boolean', '-', 'derived', 'mass <= 25 kg', 'True/False'),
    ('layer3_mode7_ok', 3, 'Mode7 above threshold', 'boolean', '-', 'derived', 'mode7_freq >= 250 Hz', 'True/False'),
    ('layer3_ratio_ok', 3, 'Frequency ratio above minimum', 'boolean', '-', 'derived', 'frequency_ratio >= 2.5', 'True/False'),
    ('layer3_pass', 3, 'Layer 3 gate pass status', 'boolean', '-', 'derived', 'all Layer 3 checks pass', 'True/False'),
    ('layer3_fail_reason', 3, 'Reason for Layer 3 failure', 'string', '-', 'derived', 'Concatenated failure reasons', 'text or empty'),
    
    # Layer 4
    ('mass_ok', 4, 'Mass constraint satisfied', 'boolean', '-', 'derived', 'mass <= 25 kg', 'True/False'),
    ('mode7_ok', 4, 'Mode7 constraint satisfied', 'boolean', '-', 'derived', 'mode7_freq >= 250 Hz', 'True/False'),
    ('ratio_ok', 4, 'Ratio constraint satisfied', 'boolean', '-', 'derived', 'frequency_ratio >= 2.5', 'True/False'),
    ('stiffness_ok', 4, 'Stiffness constraint satisfied', 'boolean', '-', 'derived', 'stiffness_proxy_k7 >= 10th percentile', 'True/False'),
    ('constraint_violation_count', 4, 'Number of violated constraints', 'int', '-', 'derived', 'count of False in [mass_ok, mode7_ok, ratio_ok, stiffness_ok]', '0-4'),
    ('passes_all_constraints', 4, 'All constraints satisfied', 'boolean', '-', 'derived', 'all constraint flags True', 'True/False'),
    ('layer4_pass', 4, 'Layer 4 gate pass status', 'boolean', '-', 'derived', 'same as passes_all_constraints', 'True/False'),
    ('layer4_fail_reason', 4, 'Reason for Layer 4 failure', 'string', '-', 'derived', 'List of violated constraints', 'text or empty'),
    
    # Layer 5
    ('min_wall_thickness_est', 5, 'Estimated minimum wall thickness', 'float', 'mm', 'mesh', 'KDTree opposite surface distance', 'varies, > 3mm typical'),
    ('sharp_edge_density', 5, 'Sharp edge density', 'float', 'edges/mm^2', 'mesh', 'edges with dihedral > 150 deg / area', 'lower is better'),
    ('fillet_feasibility_score', 5, 'Fillet feasibility score', 'float', '-', 'derived', '1 - normalized(sharp_edge_density)', '0-1, higher is better'),
    ('undercut_risk_score', 5, 'Undercut risk score', 'float', '-', 'mesh', 'Not computed (NaN)', 'NaN'),
    ('balance_uniformity', 5, 'Radial balance uniformity', 'float', '-', 'mesh', 'CV of radial distances from axis', 'lower is better'),
    ('cad_readiness_score', 5, 'CAD readiness composite score', 'float', '-', 'derived', '0.25*mesh_valid + 0.35*feat_extract + 0.40*manuf (nuanced)', '0-1, higher is better'),
    ('layer5_status', 5, 'Layer 5 gate status', 'string', '-', 'derived', 'PASS/FAIL/UNKNOWN based on cad_readiness_score', 'PASS/FAIL/UNKNOWN'),
    ('layer5_reason', 5, 'Reason for Layer 5 status', 'string', '-', 'derived', 'Explanation of status', 'text or empty'),
    
    # Layer 6
    ('normalized_mass', 6, 'Min-max normalized mass', 'float', '-', 'derived', '(mass - min) / (max - min)', '0-1'),
    ('normalized_mode7', 6, 'Min-max normalized Mode7', 'float', '-', 'derived', '(mode7 - min) / (max - min)', '0-1'),
    ('z_mass', 6, 'Z-score normalized mass', 'float', '-', 'derived', '(mass - mean) / std', 'typically -3 to 3'),
    ('z_mode7', 6, 'Z-score normalized Mode7', 'float', '-', 'derived', '(mode7 - mean) / std', 'typically -3 to 3'),
    ('z_mode11', 6, 'Z-score normalized Mode11', 'float', '-', 'derived', '(mode11 - mean) / std', 'typically -3 to 3'),
    ('z_frequency_ratio', 6, 'Z-score normalized frequency ratio', 'float', '-', 'derived', '(ratio - mean) / std', 'typically -3 to 3'),
    ('mass_margin_norm', 6, 'Normalized mass margin', 'float', '-', 'derived', '(25 - mass) / 25', '> 0 is good'),
    ('mode7_margin_norm', 6, 'Normalized Mode7 margin', 'float', '-', 'derived', '(mode7 - 250) / 250', '> 0 is good'),
    ('ratio_margin_norm', 6, 'Normalized ratio margin', 'float', '-', 'derived', '(ratio - 2.5) / 2.5', '> 0 is good'),
    ('stiffness_margin_norm', 6, 'Normalized stiffness margin', 'float', '-', 'derived', '(k7 - threshold) / threshold', '> 0 is good'),
    ('design_margin_score', 6, 'Overall design margin', 'float', '-', 'derived', 'min of all normalized margins', 'higher is safer'),
    ('risk_level', 6, 'Risk classification', 'string', '-', 'derived', 'High/Medium/Low based on percentile thresholds (P33/P67)', 'High/Medium/Low'),
    ('vehicle_class_label', 6, 'Vehicle class classification', 'string', '-', 'derived', 'Not computed (UNKNOWN)', 'UNKNOWN'),
]

# Create data dictionary dataframe
data_dict_df = pd.DataFrame(data_dict_entries, columns=[
    'column_name', 'layer', 'description', 'data_type', 'unit', 'source', 'formula_or_logic', 'expected_range_or_values'
])

# Save to CSV
data_dict_df.to_csv(OUTPUT_DATA_DICT_CSV, index=False)
print(f"Saved data dictionary to: {OUTPUT_DATA_DICT_CSV}")
print(f"  Total entries: {len(data_dict_df)}")

# Show summary by layer
print(f"\nEntries by layer:")
# Convert layer to string for sorting (handles mixed 'raw' and numeric values)
layer_counts = data_dict_df['layer'].astype(str).value_counts()
# Sort with 'raw' first, then numeric layers
layer_order = ['raw'] + [str(i) for i in range(7)]
for layer in layer_order:
    if layer in layer_counts.index:
        print(f"  Layer {layer}: {layer_counts[layer]} columns")

Creating data dictionary...
Saved data dictionary to: C:\Users\mahil.kr\GL\data-analytics-capstone\data\processed\deepwheel_data_dictionary.csv
  Total entries: 74

Entries by layer:
  Layer raw: 11 columns
  Layer 0: 8 columns
  Layer 1: 10 columns
  Layer 2: 6 columns
  Layer 3: 10 columns
  Layer 4: 8 columns
  Layer 5: 8 columns
  Layer 6: 13 columns


In [14]:
# =============================================================================
# Cell 15: Quality Report JSON
# =============================================================================

print("Generating quality report...")
print("="*60)

# Build quality report
quality_report = {
    'summary': {
        'total_designs': len(df_final),
        'designs_with_stl': int(df_final['has_stl'].sum()),
        'designs_with_step': int(df_final['has_step'].sum()),
        'designs_missing_stl': int((~df_final['has_stl']).sum()),
        'designs_missing_step': int((~df_final['has_step']).sum()),
    },
    'missing_rates': {},
    'layer_failure_rates': {},
    'top_failure_reasons': []
}

# Calculate missing rates for critical columns
critical_columns = [
    'bbox_x', 'bbox_y', 'bbox_z',
    'triangle_count', 'is_watertight',
    'symmetry_axis_confidence', 'hub_plane_flatness_proxy',
    'modal_separation', 'frequency_ratio',
    'cad_readiness_score', 'design_margin_score'
]

for col in critical_columns:
    if col in df_final.columns:
        missing_count = df_final[col].isna().sum()
        missing_pct = 100 * missing_count / len(df_final)
        quality_report['missing_rates'][col] = {
            'count': int(missing_count),
            'percent': round(missing_pct, 2)
        }

# Calculate failure rates per layer
layer_gates = {
    'layer0': 'layer0_pass',
    'layer1': 'layer1_pass',
    'layer2': 'layer2_status',
    'layer3': 'layer3_pass',
    'layer4': 'layer4_pass',
    'layer5': 'layer5_status',
}

for layer_name, gate_col in layer_gates.items():
    if gate_col in df_final.columns:
        if df_final[gate_col].dtype == bool:
            pass_count = df_final[gate_col].sum()
            fail_count = len(df_final) - pass_count
        else:
            # String status column
            pass_count = (df_final[gate_col] == 'PASS').sum()
            fail_count = (df_final[gate_col] == 'FAIL').sum()
            unknown_count = (df_final[gate_col] == 'UNKNOWN').sum()
        
        quality_report['layer_failure_rates'][layer_name] = {
            'pass_count': int(pass_count),
            'pass_percent': round(100 * pass_count / len(df_final), 2),
            'fail_count': int(fail_count),
            'fail_percent': round(100 * fail_count / len(df_final), 2)
        }
        
        if df_final[gate_col].dtype != bool:
            quality_report['layer_failure_rates'][layer_name]['unknown_count'] = int(unknown_count)
            quality_report['layer_failure_rates'][layer_name]['unknown_percent'] = round(100 * unknown_count / len(df_final), 2)

# Collect all failure reasons
fail_reason_cols = [
    'layer0_fail_reason', 'layer1_fail_reason', 'layer2_fail_reason',
    'layer3_fail_reason', 'layer4_fail_reason', 'layer5_reason'
]

all_reasons = []
for col in fail_reason_cols:
    if col in df_final.columns:
        reasons = df_final[col].dropna()
        for reason_str in reasons:
            if reason_str and reason_str.strip():
                # Split compound reasons
                for r in reason_str.split('; '):
                    if r.strip():
                        all_reasons.append(r.strip())

# Count and get top 10
reason_counts = Counter(all_reasons)
top_reasons = reason_counts.most_common(10)
quality_report['top_failure_reasons'] = [
    {'reason': reason, 'count': count}
    for reason, count in top_reasons
]

# Save to JSON
with open(OUTPUT_QUALITY_JSON, 'w') as f:
    json.dump(quality_report, f, indent=2)

print(f"Saved quality report to: {OUTPUT_QUALITY_JSON}")

# Display summary
print(f"\nQuality Report Summary:")
print(f"  Total designs: {quality_report['summary']['total_designs']}")
print(f"  With STL: {quality_report['summary']['designs_with_stl']}")
print(f"  With STEP: {quality_report['summary']['designs_with_step']}")

print(f"\nLayer Pass Rates:")
for layer, stats in quality_report['layer_failure_rates'].items():
    print(f"  {layer}: {stats['pass_percent']:.1f}% pass, {stats['fail_percent']:.1f}% fail")

print(f"\nTop 5 Failure Reasons:")
for i, item in enumerate(quality_report['top_failure_reasons'][:5], 1):
    print(f"  {i}. {item['reason']}: {item['count']} occurrences")

Generating quality report...
Saved quality report to: C:\Users\mahil.kr\GL\data-analytics-capstone\data\processed\quality_report.json

Quality Report Summary:
  Total designs: 904
  With STL: 904
  With STEP: 904

Layer Pass Rates:
  layer0: 100.0% pass, 0.0% fail
  layer1: 5.5% pass, 94.5% fail
  layer2: 39.2% pass, 60.8% fail
  layer3: 86.7% pass, 13.3% fail
  layer4: 78.0% pass, 22.0% fail
  layer5: 97.8% pass, 0.0% fail

Top 5 Failure Reasons:
  1. not_watertight: 854 occurrences
  2. low_symmetry(0.44<0.45): 218 occurrences
  3. aspect_ratio(39.3>39.3): 164 occurrences
  4. hub_not_flat(6.4>6.4mm): 131 occurrences
  5. hub_not_flat(6.5>6.4mm): 95 occurrences


In [15]:
# =============================================================================
# Cell 16: Figures for the Synopsis
# =============================================================================

print("Generating figures for synopsis...")
print("="*60)

# ----- Figure 1: Dataset Preview (Table of first 10 rows) -----
print("\nCreating Figure 1: Dataset preview...")

# Select key columns for preview
preview_cols = [
    'file_name', 'mass', 'mode7_freq', 'mode11_freq',
    'has_stl', 'layer0_pass', 'layer1_pass', 'layer4_pass',
    'risk_level', 'design_margin_score'
]
preview_data = df_final[preview_cols].head(10).copy()

# Format numeric columns
preview_data['mass'] = preview_data['mass'].apply(lambda x: f"{x:.2f}")
preview_data['mode7_freq'] = preview_data['mode7_freq'].apply(lambda x: f"{x:.1f}")
preview_data['mode11_freq'] = preview_data['mode11_freq'].apply(lambda x: f"{x:.1f}")
preview_data['design_margin_score'] = preview_data['design_margin_score'].apply(lambda x: f"{x:.3f}" if pd.notna(x) else 'NaN')

# Create figure
fig1, ax1 = plt.subplots(figsize=(16, 5))
ax1.axis('off')
ax1.set_title('DeepWheel Features Dataset - First 10 Rows Preview', fontsize=14, fontweight='bold', pad=20)

# Create table
table = ax1.table(
    cellText=preview_data.values,
    colLabels=preview_data.columns,
    cellLoc='center',
    loc='center',
    colWidths=[0.12, 0.06, 0.08, 0.08, 0.06, 0.08, 0.08, 0.08, 0.08, 0.10]
)
table.auto_set_font_size(False)
table.set_fontsize(8)
table.scale(1.2, 1.5)

# Style header
for i in range(len(preview_cols)):
    table[(0, i)].set_facecolor('#4472C4')
    table[(0, i)].set_text_props(color='white', fontweight='bold')

plt.tight_layout()
plt.savefig(FIG1_PREVIEW, dpi=150, bbox_inches='tight', facecolor='white')
plt.close()
print(f"  Saved: {FIG1_PREVIEW}")

# ----- Figure 2: Folder Tree Structure -----
print("\nCreating Figure 2: Folder tree structure...")

tree_text = """
data-analytics-capstone/
├── data/
│   ├── deepwheel_sim_results.csv    [905 designs]
│   ├── stl/                         [~850 STL mesh files]
│   ├── step/                        [~904 STEP CAD files]
│   ├── depth/                       [6249 depth images]
│   ├── rgb/                         [6249 RGB images]
│   └── processed/
│       ├── deepwheel_features_full.csv
│       ├── deepwheel_data_dictionary.csv
│       └── quality_report.json
├── docs/
│   └── figures/
│       ├── Figure1_dataset_preview.png
│       ├── Figure2_folder_tree_structure.png
│       └── Figure3_3d_wheel_preview.png
├── notebooks/
│   └── 00_preprocessing_and_feature_build.ipynb
├── QM640_Final_Synopsis_Mahil_KR.docx
└── README.md
"""

fig2, ax2 = plt.subplots(figsize=(10, 8))
ax2.axis('off')
ax2.text(0.05, 0.95, 'Repository Structure', fontsize=14, fontweight='bold', 
         transform=ax2.transAxes, verticalalignment='top')
ax2.text(0.05, 0.88, tree_text, fontsize=10, family='monospace',
         transform=ax2.transAxes, verticalalignment='top')

plt.tight_layout()
plt.savefig(FIG2_TREE, dpi=150, bbox_inches='tight', facecolor='white')
plt.close()
print(f"  Saved: {FIG2_TREE}")

# ----- Figure 3: 3D Wheel Preview -----
print("\nCreating Figure 3: 3D wheel preview...")

# Get sample STL files
sample_stls = df_final[df_final['has_stl']]['stl_path'].head(4).tolist()

if len(sample_stls) >= 1:
    try:
        n_samples = min(4, len(sample_stls))
        fig3, axes = plt.subplots(1, n_samples, figsize=(4*n_samples, 4), 
                                   subplot_kw={'projection': '3d'})
        if n_samples == 1:
            axes = [axes]
        
        for i, (ax, stl_path) in enumerate(zip(axes, sample_stls[:n_samples])):
            try:
                mesh = trimesh.load(stl_path)
                
                # Get vertices and faces
                vertices = mesh.vertices
                faces = mesh.faces
                
                # Sample faces for faster rendering
                if len(faces) > 5000:
                    face_indices = np.random.choice(len(faces), 5000, replace=False)
                    faces = faces[face_indices]
                
                # Create polygon collection
                poly3d = [[vertices[idx] for idx in face] for face in faces]
                collection = Poly3DCollection(poly3d, alpha=0.7, edgecolor='k', linewidths=0.1)
                collection.set_facecolor('#4472C4')
                ax.add_collection3d(collection)
                
                # Set axis limits
                scale = vertices.max() - vertices.min()
                center = vertices.mean(axis=0)
                ax.set_xlim(center[0] - scale/2, center[0] + scale/2)
                ax.set_ylim(center[1] - scale/2, center[1] + scale/2)
                ax.set_zlim(center[2] - scale/2, center[2] + scale/2)
                
                # Get file name
                file_name = Path(stl_path).stem
                ax.set_title(f'{file_name[:20]}...', fontsize=9)
                ax.set_xlabel('X')
                ax.set_ylabel('Y')
                ax.set_zlabel('Z')
                
            except Exception as e:
                ax.text(0.5, 0.5, 0.5, f'Error loading\n{str(e)[:30]}', 
                       ha='center', va='center', transform=ax.transAxes)
        
        plt.suptitle('Sample 3D Wheel Meshes from DeepWheel Dataset', fontsize=12, fontweight='bold')
        plt.tight_layout()
        plt.savefig(FIG3_WHEEL, dpi=150, bbox_inches='tight', facecolor='white')
        plt.close()
        print(f"  Saved: {FIG3_WHEEL}")
        
    except Exception as e:
        print(f"  Warning: Could not render 3D preview: {e}")
        print("  Creating placeholder figure...")
        
        # Create placeholder
        fig3, ax3 = plt.subplots(figsize=(10, 6))
        ax3.axis('off')
        ax3.text(0.5, 0.5, '3D Wheel Preview\n\n(Rendering not available in this environment)\n\n'
                 f'Dataset contains {df_final["has_stl"].sum()} STL files',
                 ha='center', va='center', fontsize=14, transform=ax3.transAxes)
        plt.savefig(FIG3_WHEEL, dpi=150, bbox_inches='tight', facecolor='white')
        plt.close()
        print(f"  Saved placeholder: {FIG3_WHEEL}")
else:
    print("  Warning: No STL files available for preview")

print("\nAll figures generated successfully!")

Generating figures for synopsis...

Creating Figure 1: Dataset preview...
  Saved: C:\Users\mahil.kr\GL\data-analytics-capstone\docs\figures\Figure1_dataset_preview.png

Creating Figure 2: Folder tree structure...
  Saved: C:\Users\mahil.kr\GL\data-analytics-capstone\docs\figures\Figure2_folder_tree_structure.png

Creating Figure 3: 3D wheel preview...
  Saved: C:\Users\mahil.kr\GL\data-analytics-capstone\docs\figures\Figure3_3d_wheel_preview.png

All figures generated successfully!


# What to Commit + How to Reference in Synopsis

---

## Commit Checklist

### Files to Commit

1. **Notebook**
   - `notebooks/00_preprocessing_and_feature_build.ipynb`

2. **Generated Data Files**
   - `data/processed/deepwheel_features_full.csv`
   - `data/processed/deepwheel_data_dictionary.csv`
   - `data/processed/quality_report.json`

3. **Figures**
   - `docs/figures/Figure1_dataset_preview.png`
   - `docs/figures/Figure2_folder_tree_structure.png`
   - `docs/figures/Figure3_3d_wheel_preview.png`

### Git Commands

```bash
git add notebooks/00_preprocessing_and_feature_build.ipynb
git add data/processed/
git add docs/figures/
git commit -m "Add preprocessing notebook with 7-layer feature engineering"
git push origin main
```

---

## Synopsis References

### GitHub Link

Add to your synopsis:
> The complete code and dataset are available at: https://github.com/[your-username]/data-analytics-capstone

### Data Dictionary Reference

> See **Appendix A: Data Dictionary** for complete column definitions. The full data dictionary is also available in the repository at `data/processed/deepwheel_data_dictionary.csv`.

### Figure References

- **Figure 1**: Dataset preview showing first 10 rows with key features and gate outputs
- **Figure 2**: Repository folder structure showing data organization
- **Figure 3**: Sample 3D wheel mesh renderings from the DeepWheel dataset

---

## Quality Summary for Synopsis

Include these statistics in your methodology section:

- **Total designs processed**: 905
- **Feature columns**: 60+ (across 7 validation layers)
- **Engineering gate layers**: 7 (Data Integrity, Mesh Validity, Feature Extractability, Physics Plausibility, Engineering Constraints, Manufacturability, ML Readiness)

---

## Important Disclaimers to Include

1. Manufacturability metrics are **heuristic indicators**, not certified manufacturing approval
2. Feature extractability scores are **confidence measures**, not guarantees
3. All thresholds are **conservative and documented** based on typical automotive specifications
4. Physics plausibility uses simulation outputs that should be **validated with physical testing** for production